In [ ]:
import mne

file_path = '/project/joelvoss/RAM/FR2/' #file template

#all the data subjects:
list_of_all_subjects = ['R1001P','R1016M','R1023J','R1028M','R1035M','R1050M','R1060M','R1077T','R1112M','R1177M',
                        'R1002P','R1018P','R1024E','R1030J','R1036M','R1052E','R1069M','R1085C','R1115T','R1184M',
                        'R1003P','R1020J','R1026D','R1031M','R1042M','R1053M','R1070T','R1101T','R1150J','R1006P',
                        'R1022J','R1027J','R1033D','R1048E','R1056M','R1074M','R1111M','R1176M']

important = ['events_s0.json','rec_word_eeg_s0-epo.fif','word_eeg_s0-epo.fif',
             'word_eeg_s1-epo.fif','events_s1.json','rec_word_eeg_s1-epo.fif','pairs.h5']

I first performed the permutation cluster test to understand if there is actually statistical differences in neural data between different memory task success!
Here are the resources I used to help me:
https://mne.tools/stable/generated/mne.stats.permutation_cluster_test.html
https://mne.tools/stable/generated/mne.time_frequency.tfr_morlet.html

In [ ]:
#Permutation Cluster Test on RECALLED vs NOT RECALLED on non-stimulation data

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from mne.stats import permutation_cluster_test

def power(epochs,average=False):
    frequencies = np.arange(1.5, 250, 3)
    tfr_epochs_1 = mne.time_frequency.tfr_morlet(
        epochs, n_cycles=2, return_itc=False, freqs=frequencies, decim=3, average=average
    )

    return tfr_epochs_1

def permutations(number):
    vals = np.fromiter(data_test5['is_stim'].values(), dtype=float)

    isnt_stim = vals == 0

    is_recalled = np.fromiter(data_test5['recalled'].values(), dtype=float) == 1

    is_recalled = is_recalled[:number]

    all_power = power(test_5_part1_epochs)

    for ch in np.arange(all_power.data.shape[1]): #check channels on this dimension
        F_obs, clusters, cluster_p_values, H0 = permutation_cluster_test(
            [all_power.data[(is_recalled==0), ch, :, :], all_power.data[(is_recalled==1), ch, :, :]],
            out_type="mask",
            n_permutations=1000,
            threshold=2,
            tail=0,
            seed=np.random.default_rng(seed=8675309))


for subject in list_of_all_subjects:
    no_recall = []
    recall = []

    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'):
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'
            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif')
            number = len(test_5_part1_epochs.get_data())
            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)
            print('----------------------------------------------')
            print(subject)
            permutations(number)

Here I test the hypothesis of how all individuals have the same memory networks (which they don't as seen from the AUC score of 0.51).

In [ ]:
#Train a multivariate-classifier on not stimulated data for when something is recalled or not

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from mne.stats import permutation_cluster_test
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn.utils import resample

import numpy as np
from scipy.signal import welch
from scipy.integrate import simps

def bandpower(data, band, sf=500, relative=False):
    # Use Welch's method for PSD estimation
    f, welch_psd = welch(data, sf, nperseg=len(data))  # You may adjust nperseg as needed

    # Extract the frequency range of interest
    low, high = band
    idx_band = np.logical_and(f >= low, f <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(welch_psd[idx_band], dx=f[1] - f[0])

    if relative:
        bp /= simps(welch_psd, dx=f[1] - f[0])

    return bp

def Average(lst):
    return sum(lst) / len(lst)

y = []
X = []
subject_channels = []

for subject in list_of_all_subjects:
    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'):
        print('------------------------------------------------------------------')
        print('------------------------------------------------------------------')
        print(subject)
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 0:
                    if data_test5['recalled'][test_5_directory[i]] == 0:
                        channel_data = []
                        delta = []
                        theta = []
                        alpha = []
                        beta = []
                        gamma = []
                        high_freq = []
                        for channel_number in range(len(list_of_channels)):
                            delta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[0.5,4]))
                            theta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[4,8]))
                            alpha.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[8,13]))
                            beta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[13,35]))
                            gamma.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[35,100]))
                            high_freq.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[70,200]))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        channel_data.append(Average(delta))
                        channel_data.append(Average(theta))
                        channel_data.append(Average(alpha))
                        channel_data.append(Average(beta))
                        channel_data.append(Average(gamma))
                        channel_data.append(Average(high_freq))
                        X.append(channel_data)
                        y.append(0)
                    else:
                        channel_data = []

                        delta = []
                        theta = []
                        alpha = []
                        beta = []
                        gamma = []
                        high_freq = []
                        for channel_number in range(len(list_of_channels)):
                            delta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[0.5,4]))
                            theta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[4,8]))
                            alpha.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[8,13]))
                            beta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[13,35]))
                            gamma.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[35,100]))
                            high_freq.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[70,200]))

                        channel_data.append(Average(delta))
                        channel_data.append(Average(theta))
                        channel_data.append(Average(alpha))
                        channel_data.append(Average(beta))
                        channel_data.append(Average(gamma))
                        channel_data.append(Average(high_freq))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(1)

    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'):
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 0:
                    if data_test5['recalled'][test_5_directory[i]] == 0:
                        channel_data = []
                        delta = []
                        theta = []
                        alpha = []
                        beta = []
                        gamma = []
                        high_freq = []
                        for channel_number in range(len(list_of_channels)):
                            delta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[0.5,4]))
                            theta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[4,8]))
                            alpha.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[8,13]))
                            beta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[13,35]))
                            gamma.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[35,100]))
                            high_freq.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[70,200]))

                        channel_data.append(Average(delta))
                        channel_data.append(Average(theta))
                        channel_data.append(Average(alpha))
                        channel_data.append(Average(beta))
                        channel_data.append(Average(gamma))
                        channel_data.append(Average(high_freq))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(0)
                    else:
                        channel_data = []
                        delta = []
                        theta = []
                        alpha = []
                        beta = []
                        gamma = []
                        high_freq = []
                        for channel_number in range(len(list_of_channels)):
                            delta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[0.5,4]))
                            theta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[4,8]))
                            alpha.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[8,13]))
                            beta.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[13,35]))
                            gamma.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[35,100]))
                            high_freq.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[50:-50],[70,200]))

                        channel_data.append(Average(delta))
                        channel_data.append(Average(theta))
                        channel_data.append(Average(alpha))
                        channel_data.append(Average(beta))
                        channel_data.append(Average(gamma))
                        channel_data.append(Average(high_freq))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(1)
    subject_channels.append(list_of_channels)

data = pd.DataFrame(data=np.c_[X, y], columns=['delta','theta','alpha','beta','gamma','high freq','target'])

# Assuming 'target' is your target variable column
class_counts = data['target'].value_counts()

# Find the class with the maximum count
max_class_count = class_counts.max()

# Resample each class to have the same count as the maximum class count
resampled_data = pd.DataFrame()

for class_label, count in class_counts.items():
    class_data = data[data['target'] == class_label]
    resampled_class_data = resample(class_data, replace=True, n_samples=max_class_count, random_state=42)
    resampled_data = pd.concat([resampled_data, resampled_class_data])

# Shuffle the resampled data
data = resampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

#print(data['target'])
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

# Creating a logistic regression model with L2 penalization
logreg = LogisticRegression(penalty='l2', random_state=42)

# Training the model
logreg.fit(X_train, y_train)

# Making predictions on the test set
y_pred = logreg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

y_prob = logreg.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

coefficients = logreg.coef_[0]
feature_names = X_train.columns

# Creating a DataFrame to display feature importance
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
feature_importance = feature_importance.sort_values(by='Coefficient', ascending=True)

# Displaying feature importance
print("Feature Importance:")
print(feature_importance)

print('------------------------')
print('When under the assumption that everyone has similar brain processes during memory modulation')
universal_model = logreg

Here I now adapt to each individual's unique brain pathways during memory. I get very good results!

Here is a resource I used to help me:
https://raphaelvallat.com/bandpower.html
- I also emailed with Dr. Vallat (owner of the website above) with additional questions about periodogram analysis

In [ ]:
#Train a multivariate-classifier on not stimulated data for when something is recalled or not

def pad_or_truncate(seq, max_len):
    return seq[:max_len] + [0] * (max_len - len(seq))

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

def modulation_algorithm(X, y, channels_info, return_model=False):
    final_list = []
    brain_waves = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'high freq']

    for a in channels_info:
        for b in brain_waves:
            final_list.append(b + ' ' + a)
    final_list.append('target')

    # Find the maximum length of inner lists in X
    max_len = max(len(inner_list) for inner_list in X)

    # Pad or truncate inner lists to make them consistent
    X_padded = [pad_or_truncate(inner_list, max_len) for inner_list in X]

    # Convert X (list of lists) to a 2D array
    X_array = np.array(X_padded)

    data = pd.DataFrame(data=np.c_[X_array, y], columns=final_list)

    # Assuming 'target' is your target variable column
    class_counts = data['target'].value_counts()

    # Find the class with the maximum count
    max_class_count = class_counts.max()

    # Resample each class to have the same count as the maximum class count
    resampled_data = pd.DataFrame()

    for class_label, count in class_counts.items():
        class_data = data[data['target'] == class_label]
        resampled_class_data = resample(class_data, replace=True, n_samples=max_class_count, random_state=42)
        resampled_data = pd.concat([resampled_data, resampled_class_data])

    # Shuffle the resampled data
    data = resampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

    # Creating a Random Forest Regressor model
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

    # Training the model
    rf_regressor.fit(X_train, y_train)

    # Making predictions on the test set
    y_pred = rf_regressor.predict(X_test)

    # Evaluating the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")

    # Plotting predictions vs actual values
    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Actual vs Predicted Values')
    plt.show()

    # Feature importances from Random Forest Regressor
    feature_importances = rf_regressor.feature_importances_

    # Creating a DataFrame to display feature importance
    feature_importance = pd.DataFrame({'Feature': data.columns[:-1], 'Importance': feature_importances})
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Displaying feature importance
    print("Feature Importance:")
    print(feature_importance)

    if return_model:
        return rf_regressor

algorithms_per_user = []
user_directory_for_algorithms = []

X = []
y = []
print('------------------------')
print('user-specific memory modulation prediction algorithms')

for subject in list_of_all_subjects:
    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'):
        print('------------------------------------------------------------------')
        print('------------------------------------------------------------------')
        print(subject)
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            #print(test_5_directory)
            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 0:
                    if data_test5['recalled'][test_5_directory[i]] == 0:
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(0)
                    else:
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(1)

    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'):
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 0:
                    if data_test5['recalled'][test_5_directory[i]] == 0:
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(0)
                    else:
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))
                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        X.append(channel_data)
                        y.append(1)
    print('HHHIIIII')
    if len(X) != 0:
        print('---------')
        print(subject)
        user_directory_for_algorithms.append(subject)
        print('---------')
        algorithms_per_user.append(modulation_algorithm(X, y, list_of_channels, return_model=True))
    y = []
    X = []

Now that the user-specific regression algorithms are good, I connected it towards the DBS success algorithm! Here, I determine if DBS can increase the memory performance based on the initial encoding.

In [ ]:
%%capture

#Use the multivariate-classifier trained earlier to predict in the first 0.5 seconds of stimulation data if the
#encoding state is good or bad, and then correlate these results with which brain region was stimulated and if
#that memory task was properly encoded or not

#algorithms_per_user = []
#user_directory_for_algorithms = []

def find_model(data_subject):
    for i in range(len(algorithms_per_user)):
        if user_directory_for_algorithms[i] == data_subject:
            return algorithms_per_user[i]

X = []
y = []
round_X = []
round_y = []

for user in user_directory_for_algorithms:
    #before the brain is stimulated, there is a 0.5 second duration break
    #HOWEVER 2 consecutive words always get stimulated
    count = 0
    #This means that I should just take the first 0.5 seconds of the very first word to get iEEG data before stim
    #Then I should take the last 0.5 seconds of iEEG from the brain data of the next epoch to see the effects
    #Append regressor modulation prediction on pre-stim data and post-stim data
    #Also append final result on if it was actually predicted or not

    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'):
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s0.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s0-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 1:
                    if count%2 == 0:
                        print('we')
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))

                        new_data = pd.DataFrame(channel_data).T

                        logreg = find_model(subject)
                        # Use the trained model to make predictions on the new data
                        predictions = logreg.predict(new_data)

                        round_X.append(predictions)
                        #round_X.append(universal_model.predict(new_data))

                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        #round_X.append(channel_data)
                    else:
                        print('are')
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[70,200]))

                        new_data = pd.DataFrame(channel_data).T

                        logreg = find_model(subject)
                        # Use the trained model to make predictions on the new data
                        predictions = logreg.predict(new_data)

                        round_X.append(predictions)
                        #round_X.append(universal_model.predict(new_data))

                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        #round_X.append(channel_data)
                        round_y.append(data_test5['recalled'][test_5_directory[i]])
                        round_y.append(subject)

                        X.append(round_X)
                        y.append(round_y)

                        round_X = []
                        round_y = []

                    count += 1

    if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'):
        if os.path.isfile('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif'):
            events_file_path = '/project/joelvoss/RAM/FR2/' + subject + '/events_s1.json'

            with open(events_file_path, 'r') as file:
                data_test5 = json.load(file)

            test_5_dict = {'type': {str(k): v for k, v in data_test5['type'].items() if (v == 'WORD')}} #want type==WORD and is_stim==0
            test_5_directory = list(test_5_dict['type'].keys())

            test_5_part1_epochs = mne.read_epochs('/project/joelvoss/RAM/FR2/' + subject + '/word_eeg_s1-epo.fif')

            list_of_channels = list(test_5_part1_epochs.ch_names)
            for i in range(len(test_5_directory)):
                if data_test5['is_stim'][test_5_directory[i]] == 1:
                    if count%2 == 0:
                        print('we')
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[0:250],[70,200]))

                        new_data = pd.DataFrame(channel_data).T

                        logreg = find_model(subject)
                        # Use the trained model to make predictions on the new data
                        predictions = logreg.predict(new_data)

                        round_X.append(predictions)
                        #round_X.append(universal_model.predict(new_data))

                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        #round_X.append(channel_data)
                    else:
                        print('are')
                        channel_data = []
                        for channel_number in range(len(list_of_channels)):
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[0.5,4]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[4,8]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[8,13]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[13,35]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[35,100]))
                            channel_data.append(bandpower(test_5_part1_epochs[i].get_data()[0][channel_number].flatten()[-250:],[70,200]))

                        new_data = pd.DataFrame(channel_data).T

                        logreg = find_model(subject)
                        # Use the trained model to make predictions on the new data
                        predictions = logreg.predict(new_data)

                        round_X.append(predictions)
                        #round_X.append(universal_model.predict(new_data))

                        #FFT
                        #delta 0.5-4, theta 4-8, alpha 8-13, beta 13-35, gamma 35-100, high freq 70-200
                        #round_X.append(channel_data)
                        round_y.append(data_test5['recalled'][test_5_directory[i]])
                        round_y.append(subject)

                        X.append(round_X)
                        y.append(round_y)

                        round_X = []
                        round_y = []

                    count += 1

saved_X = X
saved_y = y

for a in range(len(saved_X)):
    saved_X[a] = [saved_X[a][0][0],saved_X[a][1][0]]

for a in range(len(saved_y)):
    saved_y[a] = saved_y[a][0]

data = pd.DataFrame(data=np.c_[saved_X, saved_y], columns=['user specific prediction prestimulus','user specific prediction poststimulus','target'])


Displayed AUC and Confusion Matrix:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.utils import resample

# Assuming 'target' is your target variable column
class_counts = data['target'].value_counts()

# Find the class with the maximum count
max_class_count = class_counts.max()

# Resample each class to have the same count as the maximum class count
resampled_data = pd.DataFrame()

for class_label, count in class_counts.items():
    class_data = data[data['target'] == class_label]
    resampled_class_data = resample(class_data, replace=True, n_samples=max_class_count, random_state=42)
    resampled_data = pd.concat([resampled_data, resampled_class_data])

# Shuffle the resampled data
data = resampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Recalculate X using only the 1st feature of data
X = data.iloc[:, 0].values.reshape(-1, 1)
y = data['target']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Training the model
rf_classifier.fit(X_train, y_train)

# Making predictions on the test set
y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]  # Probability of positive class

# Computing ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Plotting the ROC curve with shading
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.fill_between(fpr, 0, tpr, color='blue', alpha=0.3)
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--', alpha=0.7)

# Save the ROC curve plot
plt.savefig('roc_curve.png', dpi=300)
plt.show()

# Making binary predictions based on a threshold (you can choose a threshold based on your preference)
threshold = 0.5
y_pred_binary = (y_pred_proba > threshold).astype(int)

# Plotting the confusion matrix with actual numbers
conf_matrix = confusion_matrix(y_test, y_pred_binary)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', cbar=False,
            xticklabels=['Predicted No Recall', 'Predicted Recall'], yticklabels=['Actual No Recall', 'Actual Recall'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

I also graphed the effects of DBS before and after in different memory encoding states


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

# Assuming 'target' is your target variable column
class_counts = data['target'].value_counts()

# Find the class with the maximum count
max_class_count = class_counts.max()

# Resample each class to have the same count as the maximum class count
resampled_data = pd.DataFrame()

for class_label, count in class_counts.items():
    class_data = data[data['target'] == class_label]
    resampled_class_data = resample(class_data, replace=True, n_samples=max_class_count, random_state=42)
    resampled_data = pd.concat([resampled_data, resampled_class_data])

# Shuffle the resampled data
data = resampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Recalculate X using the 1st and 2nd features of data
X = data.iloc[:, :2]

# Plotting the points with grid lines and y=x line
plt.figure(figsize=(12, 8))  # Adjust the figsize to your preference
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], label='Data Points', alpha=0.7)

# Adding grid lines
plt.grid(True, linestyle='--', alpha=0.7)

# Adding y=x line
plt.plot([X.iloc[:, 0].min(), X.iloc[:, 1].max()], [X.iloc[:, 0].min(), X.iloc[:, 1].max()], color='black', linestyle='--', label='y=x')

# Adding labels and title
plt.xlabel('pre-stimulus memory modulation prediction')
plt.ylabel('post-stimulus memory modulation prediction')
plt.title('Effects of Brain Stimulation on Memory Modulation State')

# Displaying the plot
plt.legend()
plt.show()

Trying to do line of best fit between pre-stimulus memory encoding state and recall success

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# Assuming 'target' is your target variable column
class_counts = data['target'].value_counts()

# Find the class with the maximum count
max_class_count = class_counts.max()

# Resample each class to have the same count as the maximum class count
resampled_data = pd.DataFrame()

for class_label, count in class_counts.items():
    class_data = data[data['target'] == class_label]
    resampled_class_data = resample(class_data, replace=True, n_samples=max_class_count, random_state=42)
    resampled_data = pd.concat([resampled_data, resampled_class_data])

# Shuffle the resampled data
data = resampled_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Plotting the points
plt.figure(figsize=(12, 8))  # Adjust the figsize to your preference
plt.scatter(data.iloc[:, 0], data['target'], label='Data Points', alpha=0.7)

# Fitting a polynomial curve to the data
def func(x, a, b, c):
    return a * x**2 + b * x + c

params, _ = curve_fit(func, data.iloc[:, 0], data['target'])
x_fit = np.linspace(data.iloc[:, 0].min(), data.iloc[:, 0].max(), 100)
y_fit = func(x_fit, *params)

# Plotting the best fit curve
plt.plot(x_fit, y_fit, color='red', label=f'Best Fit Curve: {params[0]:.2f}x^2 + {params[1]:.2f}x + {params[2]:.2f}')

# Adding labels and title
plt.xlabel('prestimulus memory modulation prediction')
plt.xlim(0,1)
plt.ylabel('recall success')
plt.ylim(0,1)
plt.title('Scatter Plot with Best Fit Curve')

# Displaying the plot
plt.legend()
plt.show()
